In [1]:
# Day Eleven
import operator
data = [x.strip() for x in open('day_11.txt')] 
test_data = [x.strip() for x in open('test.txt')] 

def print_map(seat_map, rows):
    for y in range(rows):
        pl = ''
        for s in seat_map:
            if s[1] == y:
                pl += seat_map[s]
        print(pl)
    print('0000000000')
    
def init_seat_map(data):
    seat_map = dict()
    y = -1
    for line in data:
        y += 1
        for x in range(len(line)):
            seat_map[(x,y)] = line[x]
    return seat_map

def get_adj_seats(seat, seat_map, cycle):
    transforms = [(-1,-1), (0,-1), (1,-1), (-1,0), (1,0), (-1,1), (0,1), (1,1)]
    adjacent = list()
    for t in transforms:
        ns = tuple(map(operator.add, seat, t))
        if ns in seat_map and seat_map[ns] != '.':
            adjacent.append(seat_map[ns])
        elif cycle == 1 and ns in seat_map: #if second rule, keep searching
            done = False
            while not done:
                ns = tuple(map(operator.add, ns, t))
                if ns not in seat_map:
                    done = True
                elif seat_map[ns] != '.':
                    adjacent.append(seat_map[ns])
                    done = True
    return adjacent
    
stable = 0
cycle = -1
data_in_use = data
while stable < 2:
    if cycle != stable:
        seat_map = init_seat_map(data_in_use)
        cycle += 1
    new_map = seat_map.copy() #start this way since many un-changed
    for seat in seat_map:
        if seat_map[seat] != '.': #if not floor
            adjacent = get_adj_seats(seat, seat_map, cycle)
            if seat_map[seat] == 'L': #if empty
                if adjacent.count('#') == 0: #if no adj occupied, becomes occupied
                    new_map[seat] = '#'
            elif seat_map[seat] == '#': #if occupied
                thresh = 4 if cycle == 0 else 5
                if adjacent.count('#') >= thresh: #if 4+ adj occupied, becomes empty
                    new_map[seat] = 'L'
#     print_map(seat_map, 10)
#     print_map(new_map, 10)
#     print(new_map == seat_map)
    if new_map == seat_map:
        print(list(seat_map.values()).count('#'))
        stable += 1
    else:
        seat_map = new_map.copy()
        

2183
1990


In [2]:
# Day Twelve
import numpy as np
data = [x.strip() for x in open('day_12.txt')] 
test_data = [x.strip() for x in open('test.txt')]

def facing_to_compass(facing):
    facing_map = {0:'E', 90:'S', 180:'W', 270:'N'}
    return facing_map[facing]

def rotate(L_R, facing, degrees):
    if L_R == 'L':
        degrees = degrees*-1
    facing += degrees
    if facing >= 360:
        facing -= 360
    elif facing < 0:
        facing += 360
    return facing

def rot_90(L_R, waypoint):
    if L_R == 'R':
        new_waypoint = np.array([waypoint[1], waypoint[0]*-1]) #x'=y, y'=-x
    elif L_R == 'L':
        new_waypoint = np.array([waypoint[1]*-1, waypoint[0]]) #x'=-y, y'=x
    return new_waypoint

def rot_waypoint(L_R, waypoint, degrees):
    num_turns = degrees // 90 #number of times to repeat 90 degree turn
    for i in range(num_turns):
        waypoint = rot_90(L_R, waypoint)
    return waypoint

def move(pos, action, magnitude):
    action_map = {'N':np.array([0,1]), 'S':np.array([0,-1]), 'E':np.array([1,0]), 'W':np.array([-1,0])}
    movement = action_map[action]*magnitude
    return pos + movement

def move_to_waypoint(pos2, waypoint, magnitude):
    movement = waypoint*magnitude
    return pos2 + movement

def parse_data(data):
    pos = np.array([0,0])
    pos2 = np.array([0,0]) #for pt2
    waypoint = np.array([10,1]) #for pt2
    facing = 0 #zero is due east
    for line in data:
        action, magnitude = line[0], int(line[1:])
        if action == 'L' or action == 'R':
            facing = rotate(action, facing, magnitude)
            waypoint = rot_waypoint(action, waypoint, magnitude)
        elif action == 'F':
            curr_direction = facing_to_compass(facing) #convert to N,S,E,W for pt1
            pos = move(pos, curr_direction, magnitude)
            pos2 = move_to_waypoint(pos2, waypoint, magnitude)
        else:
            pos = move(pos, action, magnitude)
            waypoint = move(waypoint, action, magnitude)
    return pos, pos2

pos, pos2 = parse_data(data)
pt1 = np.absolute(pos).sum() #calc manhattan
pt2 = np.absolute(pos2).sum() #calc manhattan
print(f'pt1: {pt1}, pt2: {pt2}')

pt1: 445, pt2: 42495


In [255]:
%%time
# Day 13
import numpy as np
from math import gcd # Python versions 3.5 and above
from functools import reduce # Python version 3.x

def lcm(denominators):
    return reduce(lambda a,b: a*b // gcd(a,b), denominators)

def step(seed, step_size, next_mod, next_bus):
    for n in range(seed, 9999999999999999, step_size):
        if n % next_bus == next_mod:
            return n

def find_magic_t(raw_ids, filtered_ids):
    descending_ids = sorted(filtered_ids, reverse=True)
    #these are the key values
    #the lowest number for each bus (incl zero) for which resultant offset will be as specified
    #take bus_id - offset, but include mod operator for the zero case, i.e. first bus
    desired_mods = np.array([(i - raw_ids.index(str(i)))%i for i in descending_ids])
    
    seed = desired_mods[0] #start at (largest_bus - offset)
    buses_in_phase = list()
    for i in range(len(descending_ids)-1):
        this_bus, next_bus = descending_ids[i:i+2]
        buses_in_phase.append(this_bus)
        step_size = lcm(buses_in_phase) #can straight multiply if you know all #s are co-prime
        next_mod = desired_mods[i+1] #req'd mod on next bus
#         print(this_bus, next_bus, next_mod, step_size, buses_in_phase)
#         step buses 'into phase' with one another, one at a time
#         start with the two largest buses, work down
#         because the larger the [cumulative] phase the larger each step we can take
        seed = step(seed, step_size, next_mod, next_bus)
    
    return seed

def run(my_file):
    data = [x.strip() for x in open(my_file)] 
    my_arrival = int(data[0])
    bus_ids = data[1].split(',')
    filtered_ids = list(map(int, [b for b in bus_ids if b != 'x'])) #filter for numbers only
    pt2 = find_magic_t(bus_ids, filtered_ids) #solve pt2
    wait_time_per_bus = [(bus - my_arrival % bus, bus) for bus in filtered_ids]
    my_wait, my_bus = min(wait_time_per_bus)
    pt1 = my_bus * my_wait
    print(f'pt1: {pt1}, pt2: {pt2}')

# my_file = 'test.txt'
my_file = 'day_13.txt'
run(my_file)

pt1: 2305, pt2: 552612234243498
Wall time: 1e+03 µs


In [290]:
# Day 14
def num_to_bin(num):
    return format(num, '036b')
def bin_to_num(bin):
    return int(bin, 2)

def apply_mask(mask, binary):
    res = list(binary)
    for i in range(len(mask)):
        if mask[i] != 'X':
            res[i] = mask[i]
    return ''.join(res)

def apply_mask2(mask, address):
    binary_address = num_to_bin(address)
    floaters = list()
    res = list(binary_address)
    for i in range(len(mask)):
        if mask[i] == 'X':
            floaters.append(i)
            res[i] = '0' #we will write all floating bits as '0' first, making copies with '1' later
        elif mask[i] == '1':
            res[i] = '1'
    strs_to_write = [''.join(res)] #initialize the list of str representations of addresses
    for f in floaters:
        queue = list()
        for s in strs_to_write: #by making a copy of all pending strings, we double the total addresses per floater
            temp = list(s)
            temp[f] = '1'
            queue.append(''.join(temp))
        strs_to_write.extend(queue)
    addresses_to_write = [bin_to_num(s) for s in strs_to_write] #convert addresses from bin to int
    return addresses_to_write

def write_mem2(address, mask, val_to_store):
    addresses_to_write = apply_mask2(mask, address)
    for a in addresses_to_write:
        mem2[a] = val_to_store

def run(my_file):
    data = [x.strip() for x in open(my_file)]
    for line in data:
        head, tail = line.split(' = ')
        if head == 'mask':
            mask = tail
        else:
            address = int(head.strip('mem[]'))
            binary = num_to_bin(int(tail))
            masked_binary = apply_mask(mask, binary)
            mem[address] = bin_to_num(masked_binary)
            write_mem2(address, mask, int(tail))
    pt1, pt2 = sum(mem.values()), sum(mem2.values())
    print(f'pt1: {pt1}, pt2: {pt2}')

mem = dict()
mem2 = dict()
# my_file = 'test.txt'
my_file = 'day_14.txt'
run(my_file)

pt1: 9628746976360, pt2: 4574598714592
Wall time: 133 ms
